# Kepler Framework, Emulate noringer

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy
import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = 'output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = 'output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = 'output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

## Setup Chains:

In [4]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_noringer_L1EM15VHI" , L1Seed = 'L1_EM15VHI'),
            Chain( "HLT_e28_lhtight_nod0_noringer_ivarloose" , L1Seed = 'L1_EM22VHI'),
            Chain( "HLT_e60_lhmedium_nod0_noringer_L1EM24VHI" , L1Seed = 'L1_EM24VHI'),
            Chain( "HLT_e140_lhloose_nod0_noringer"  , L1Seed = 'L1_EM24VHI'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'trig_L2_cl_vloose_et12to22')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'trig_L2_cl_tight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'trig_L2_cl_medium_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'trig_L2_cl_loose_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [8]:
data17_df = load_hdf(dpath_data17)

In [9]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:21<00:00, 20.42s/it]


In [10]:
data17_df.shape

(43311283, 103)

### Validate:

In [11]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_noringer.root')

2022-02-03 20:24:05,199 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_noringer.root


In [12]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_noringer_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_noringer_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_noringer_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_noringer'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:30<00:00, 18.19s/it]


HLT_e17_lhvloose_nod0_noringer_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.4788  | 29957673/32047563 |
|   L2   | 93.2579  | 29886887/32047563 |
| EFCalo | 91.9467  | 29466662/32047563 |
|  HLT   | 89.9275  | 28819560/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:05<00:00, 13.15s/it]


HLT_e28_lhtight_nod0_noringer_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 93.9991  | 21399463/22765602 |
|   L2   | 93.8887  | 21374319/22765602 |
| EFCalo | 92.9700  | 21165186/22765602 |
|  HLT   | 86.8600  | 19774199/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


HLT_e60_lhmedium_nod0_noringer_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.3246  | 1331192/1367786 |
|   L2   | 97.2366  | 1329988/1367786 |
| EFCalo | 95.1484  | 1301426/1367786 |
|  HLT   | 91.8844  | 1256782/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 25.03it/s]

HLT_e140_lhloose_nod0_noringer
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 98.1566  | 59425/60541  |
|   L2   | 98.0856  | 59382/60541  |
| EFCalo | 96.7543  | 58576/60541  |
|  HLT   | 94.7094  | 57338/60541  |
+--------+----------+--------------+


In [13]:
eff_data17_Zee.save()

In [14]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_noringer.root')

2022-02-03 20:27:28,884 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_noringer.root


In [15]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_noringer_L1EM15VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_noringer_ivarloose' , reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_noringer_L1EM24VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_noringer'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:20<00:00,  4.19s/it]


HLT_e17_lhvloose_nod0_noringer_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo | 35.8232  | 3707702/10349992 |
|   L2   | 35.7604  | 3701194/10349992 |
| EFCalo | 35.3166  | 3655262/10349992 |
|  HLT   |  0.2862  |  29618/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.59s/it]


HLT_e28_lhtight_nod0_noringer_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo | 26.4611  | 2448794/9254300 |
|   L2   | 26.3965  | 2442814/9254300 |
| EFCalo | 26.0071  | 2406771/9254300 |
|  HLT   |  0.0034  |   313/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.19s/it]


HLT_e60_lhmedium_nod0_noringer_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo | 34.4354  | 2703931/7852181 |
|   L2   | 34.3616  | 2698132/7852181 |
| EFCalo | 33.8804  | 2660352/7852181 |
|  HLT   |  0.0059  |   463/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.78s/it]

HLT_e140_lhloose_nod0_noringer
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo | 94.5886  | 5329159/5634041 |
|   L2   | 69.0642  | 3891106/5634041 |
| EFCalo | 67.8555  | 3823006/5634041 |
|  HLT   |  0.0715  |   4031/5634041  |
+--------+----------+-----------------+


In [16]:
eff_data17_JF17.save()

In [17]:
del data17_df

## Emulate 2018:

In [18]:
data18_df = load_hdf(dpath_data18)

In [19]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:20<00:00, 20.09s/it]


In [20]:
data18_df.shape

(41873973, 103)

In [21]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_noringer.root')

2022-02-03 20:31:57,535 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_noringer.root


In [22]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_noringer_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_noringer_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_noringer_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_noringer'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:30<00:00, 18.14s/it]


HLT_e17_lhvloose_nod0_noringer_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5054  | 30501925/32620495 |
|   L2   | 93.2911  | 30432034/32620495 |
| EFCalo | 91.9563  | 29996605/32620495 |
|  HLT   | 90.4521  | 29505934/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:05<00:00, 13.18s/it]


HLT_e28_lhtight_nod0_noringer_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 93.9523  | 21903884/23313826 |
|   L2   | 93.8582  | 21881941/23313826 |
| EFCalo | 93.0651  | 21697027/23313826 |
|  HLT   | 87.3949  | 20375096/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


HLT_e60_lhmedium_nod0_noringer_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.3201  | 1344648/1381675 |
|   L2   | 97.2357  | 1343481/1381675 |
| EFCalo | 95.1497  | 1314660/1381675 |
|  HLT   | 92.1587  | 1273334/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 24.84it/s]

HLT_e140_lhloose_nod0_noringer
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.1934  | 59298/60389  |
|   L2   | 98.0891  | 59235/60389  |
| EFCalo | 96.9100  | 58523/60389  |
|  HLT   | 95.0024  | 57371/60389  |
+--------+----------+--------------+


In [23]:
eff_data18_Zee.save()

In [24]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_noringer.root')

2022-02-03 20:35:21,559 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_noringer.root


In [25]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_noringer_L1EM15VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_noringer_ivarloose' , reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_noringer_L1EM24VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_noringer'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.40s/it]


HLT_e17_lhvloose_nod0_noringer_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo | 36.7962  | 3115568/8467098 |
|   L2   | 36.7373  | 3110583/8467098 |
| EFCalo | 36.3002  | 3073571/8467098 |
|  HLT   |  1.0214  |  86482/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:14<00:00,  2.95s/it]


HLT_e28_lhtight_nod0_noringer_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo | 27.2934  | 2082161/7628804 |
|   L2   | 27.2326  | 2077521/7628804 |
| EFCalo | 26.8566  | 2048840/7628804 |
|  HLT   |  0.0609  |   4647/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.61s/it]


HLT_e60_lhmedium_nod0_noringer_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo | 34.9747  | 2295036/6561985 |
|   L2   | 34.9053  | 2290483/6561985 |
| EFCalo | 34.4419  | 2260071/6561985 |
|  HLT   |  0.0792  |   5198/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:11<00:00,  2.29s/it]

HLT_e140_lhloose_nod0_noringer
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo | 94.6509  | 4521018/4776518 |
|   L2   | 64.7454  | 3092574/4776518 |
| EFCalo | 63.6410  | 3039826/4776518 |
|  HLT   |  0.4256  |  20331/4776518  |
+--------+----------+-----------------+


In [26]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [5]:
mc16_df = load_hdf(dpath_mc16)

In [6]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 34.21it/s]


In [7]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_noringer.root')

2022-02-08 20:00:17,250 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_noringer.root


In [8]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_noringer_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_noringer_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_noringer_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_noringer'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.69it/s]


HLT_e17_lhvloose_nod0_noringer_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 92.3517  | 72606/78619  |
|   L2   | 92.3454  | 72601/78619  |
| EFCalo | 72.7407  | 57188/78619  |
|  HLT   | 71.7002  | 56370/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.59it/s]


HLT_e28_lhtight_nod0_noringer_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 92.4082  | 67555/73105  |
|   L2   | 92.4054  | 67553/73105  |
| EFCalo | 72.8185  | 53234/73105  |
|  HLT   | 69.9993  | 51173/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.36it/s]


HLT_e60_lhmedium_nod0_noringer_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 92.4288  | 68340/73938  |
|   L2   | 92.4261  | 68338/73938  |
| EFCalo | 72.3944  | 53527/73938  |
|  HLT   | 70.5862  | 52190/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 21.96it/s]

HLT_e140_lhloose_nod0_noringer
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 91.6386  | 62273/67955  |
|   L2   | 91.6386  | 62273/67955  |
| EFCalo | 71.5576  | 48627/67955  |
|  HLT   | 70.1273  | 47655/67955  |
+--------+----------+--------------+


In [9]:
eff_mc16.save()